In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import os
path = "/cluster/home/mlindner/Github/master_thesis_project/src/data/outputs"
plots_path = "/cluster/home/mlindner/Github/master_thesis_project/src/data/outputs/plots/by_model/"
os.makedirs(plots_path, exist_ok=True)

In [11]:
file_name = "250417.csv"
df = pd.read_csv(os.path.join(path, file_name))

model_names = [
    "deepseek-chat", 
    "claude-3-haiku-20240307", 
    "gemini-2.0-flash-lite", 
    #"gemini-1.5-flash-002", 
    "gpt-4o-mini-2024-07-18", 
    #"gemini-2.0-flash", 
    "gemini-2.5-pro-exp-03-25", 
    "gpt-4.1-mini-2025-04-14", 
    "gpt-3.5-turbo"
]
# Filter the DataFrame to include only the specified model names
df = df[df['model_name_1'].isin(model_names) | df['model_name_2'].isin(model_names)]
df['agent_1_scores'] = df['agent_1_scores'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['agent_2_scores'] = df['agent_2_scores'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['agent_1_cumulative_scores'] = df['agent_1_scores'].apply(lambda x: [0] + [sum(x[:i+1]) for i in range(len(x))] if isinstance(x, list) else x) 
df['agent_2_cumulative_scores'] = df['agent_2_scores'].apply(lambda x: [0] + [sum(x[:i+1]) for i in range(len(x))] if isinstance(x, list) else x)

#parse as lists, something went wrong when saving the csv
df['truthful_agent_1'] = df['truthful_agent_1'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['truthful_agent_2'] = df['truthful_agent_2'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
# Parse the actions as lists of strings
df['agent_1_actions'] = df['agent_1_actions'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['agent_2_actions'] = df['agent_2_actions'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['agent_1_messages'] = df['agent_1_messages'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['agent_2_messages'] = df['agent_2_messages'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['intent_agent_1'] = df['intent_agent_1'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['intent_agent_2'] = df['intent_agent_2'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df = df.rename(columns={'game_name': 'GameName'})

# Create a new DataFrame with one row per agent
df_agent1 = df.copy()
df_agent2 = df.copy()

# Add a column to indicate the agent
df_agent1['Agent'] = 'Agent 1'
df_agent2['Agent'] = 'Agent 2'
# Rename columns to remove agent-specific prefixes
df_agent1 = df_agent1.rename(columns={
    'personality_1': 'Personality',
    'agent_1_scores': 'Scores',
    'agent_1_messages': 'Messages',
    'agent_1_actions': 'Actions',
    'agent_1_cumulative_scores': 'CumulativeScores',
    'intent_agent_1': 'Intent',
    'truthful_agent_1': 'Truthful',
    'analysis_agent_1': 'Analysis',
    'model_name_1': 'ModelName',
    'model_provider_1': 'ModelProvider',
})

df_agent2 = df_agent2.rename(columns={
    'personality_2': 'Personality',
    'agent_2_scores': 'Scores',
    'agent_2_messages': 'Messages',
    'agent_2_actions': 'Actions',
    'agent_2_cumulative_scores': 'CumulativeScores',
    'intent_agent_2': 'Intent',
    'truthful_agent_2': 'Truthful',
    'analysis_agent_2': 'Analysis',
    'model_name_2': 'ModelName',
    'model_provider_2': 'ModelProvider',
})

# Remove the other columns
df_agent1 = df_agent1.drop(columns=[
    'personality_2', 'agent_2_scores', 'agent_2_messages', 'agent_2_actions', 
    'agent_2_cumulative_scores', 'intent_agent_2', 'truthful_agent_2', 'analysis_agent_2',
    'model_name_2', 'model_provider_2'
])
df_agent2 = df_agent2.drop(columns=[
    'personality_1', 'agent_1_scores', 'agent_1_messages', 'agent_1_actions', 
    'agent_1_cumulative_scores', 'intent_agent_1', 'truthful_agent_1', 'analysis_agent_1',
    'model_name_1', 'model_provider_1'
])

# Combine the two DataFrames
df_agents = pd.concat([df_agent1, df_agent2], ignore_index=True)

nonmbti = ['NONE', 'ALTRUISTIC', 'SELFISH']
# Map each personality to its dichotomies
df_agents['I/E'] = df_agents['Personality'].apply(lambda x: 'I' if x[0] == 'I' else 'E' if x not in nonmbti else None)
df_agents['N/S'] = df_agents['Personality'].apply(lambda x: 'N' if x[1] == 'N' else 'S' if x not in nonmbti else None)
df_agents['T/F'] = df_agents['Personality'].apply(lambda x: 'T' if x[2] == 'T' else 'F' if x not in nonmbti else None)
df_agents['J/P'] = df_agents['Personality'].apply(lambda x: 'J' if x[3] == 'J' else 'P' if x not in nonmbti else None)

df_agents["TotalScore"] = df_agents["CumulativeScores"].apply(lambda x: x[-1])
df_agents["Truthfullness"] = df_agents["Truthful"].apply(lambda x: sum(x)/len(x) if len(x) > 0 else 0)

#total cost USD
print(df_agents['total_cost_USD'].sum())

df_agents.head()
#df_agents.to_csv("/cluster/home/mlindner/Github/master_thesis_project/src/data/outputs/plots/stability_PD/stability_PD.csv", index=False)

In [12]:
sns.set(style="whitegrid")
sns.set_context("talk")
sns.set_palette("husl")
fig, axs = plt.subplots(2, 2, figsize=(15, 10))
sns.violinplot(x='I/E', y='TotalScore', data=df_agents, ax=axs[0, 0])
sns.violinplot(x='N/S', y='TotalScore', data=df_agents, ax=axs[0, 1])
sns.violinplot(x='T/F', y='TotalScore', data=df_agents, ax=axs[1, 0])
sns.violinplot(x='J/P', y='TotalScore', data=df_agents, ax=axs[1, 1])
plt.tight_layout()
plt.savefig(plots_path + "score_violin_per_dichotomy.png")
plt.show()


In [13]:
# Prepare the data for plotting
df_agents['Round'] = df_agents['CumulativeScores'].apply(lambda x: list(range(len(x))))
df_agents['TruthfulByRound'] = df_agents.apply(lambda row: list(zip(row['Round'], row['Truthful'])), axis=1)

# Explode the data to have one row per round
df_exploded = df_agents.explode('TruthfulByRound')
df_exploded['Round'] = df_exploded['TruthfulByRound'].apply(lambda x: x[0] if isinstance(x, tuple) else None)
df_exploded['Truthful'] = df_exploded['TruthfulByRound'].apply(lambda x: x[1] if isinstance(x, tuple) else None)

# Filter out rows with missing values
df_exploded = df_exploded.dropna(subset=['Round', 'Truthful'])

# Convert Round to integer for proper sorting
df_exploded['Round'] = df_exploded['Round'].astype(int)

# Plot average truthfulness by round for each dichotomy
sns.set(style="whitegrid")
sns.set_context("talk")
sns.set_palette("husl")
fig, axs = plt.subplots(2, 2, figsize=(15, 10))

sns.lineplot(data=df_exploded, x='Round', y='Truthful', hue='I/E', ax=axs[0, 0])
axs[0, 0].set_title('Average Truthfulness by Round - I/E')

sns.lineplot(data=df_exploded, x='Round', y='Truthful', hue='N/S', ax=axs[0, 1])
axs[0, 1].set_title('Average Truthfulness by Round - N/S')

sns.lineplot(data=df_exploded, x='Round', y='Truthful', hue='T/F', ax=axs[1, 0])
axs[1, 0].set_title('Average Truthfulness by Round - T/F')

sns.lineplot(data=df_exploded, x='Round', y='Truthful', hue='J/P', ax=axs[1, 1])
axs[1, 1].set_title('Average Truthfulness by Round - J/P')

plt.tight_layout()
plt.savefig(plots_path + "average_truthfulness_by_round_dichotomy.png")
plt.show()

In [ ]:
# Prepare the data for plotting by model
#sns.set(style="whitegrid")
#sns.set_context("talk")
#sns.set_palette("husl")
#
#for model_name in model_names:
#    model_data = df_agents[df_agents['ModelName'] == model_name]
#    model_data['Round'] = model_data['CumulativeScores'].apply(lambda x: list(range(len(x))))
#    model_data['TruthfulByRound'] = model_data.apply(lambda row: list(zip(row['Round'], row['Truthful'])), axis=1)
#
#    # Explode the data to have one row per round
#    model_exploded = model_data.explode('TruthfulByRound')
#    model_exploded['Round'] = model_exploded['TruthfulByRound'].apply(lambda x: x[0] if isinstance(x, tuple) else None)
#    model_exploded['Truthful'] = model_exploded['TruthfulByRound'].apply(lambda x: x[1] if isinstance(x, tuple) else None)
#
#    # Filter out rows with missing values
#    model_exploded = model_exploded.dropna(subset=['Round', 'Truthful'])
#
#    # Convert Round to integer for proper sorting
#    model_exploded['Round'] = model_exploded['Round'].astype(int)
#
#    # Create subplots for each dichotomy
#    fig, axs = plt.subplots(2, 2, figsize=(15, 10))
#    fig.suptitle(f"Average Truthfulness by Round for {model_name}", fontsize=16)
#
#    sns.lineplot(data=model_exploded, x='Round', y='Truthful', hue='I/E', ax=axs[0, 0])
#    axs[0, 0].set_title('I/E Dichotomy')
#
#    sns.lineplot(data=model_exploded, x='Round', y='Truthful', hue='N/S', ax=axs[0, 1])
#    axs[0, 1].set_title('N/S Dichotomy')
#
#    sns.lineplot(data=model_exploded, x='Round', y='Truthful', hue='T/F', ax=axs[1, 0])
#    axs[1, 0].set_title('T/F Dichotomy')
#
#    sns.lineplot(data=model_exploded, x='Round', y='Truthful', hue='J/P', ax=axs[1, 1])
#    axs[1, 1].set_title('J/P Dichotomy')
#
#    plt.tight_layout(rect=[0, 0, 1, 0.95])
#    plt.savefig(plots_path + f"average_truthfulness_by_round_dichotomy_{model_name}.png")
#    plt.show()

In [ ]:
# Create violin plots for Truthfullness across individual dichotomies
sns.set(style="whitegrid")
sns.set_context("talk")
sns.set_palette("husl")
fig, axs = plt.subplots(2, 2, figsize=(15, 10))

sns.violinplot(x='I/E', y='Truthfullness', data=df_agents, ax=axs[0, 0], hue='I/E')
axs[0, 0].axhline(df_agents[df_agents['I/E'] == 'I']['Truthfullness'].mean(), color='blue', linestyle='--', label='Mean I')
axs[0, 0].axhline(df_agents[df_agents['I/E'] == 'E']['Truthfullness'].mean(), color='orange', linestyle='--', label='Mean E')

sns.violinplot(x='N/S', y='Truthfullness', data=df_agents, ax=axs[0, 1], hue='N/S')
axs[0, 1].axhline(df_agents[df_agents['N/S'] == 'N']['Truthfullness'].mean(), color='blue', linestyle='--', label='Mean N')
axs[0, 1].axhline(df_agents[df_agents['N/S'] == 'S']['Truthfullness'].mean(), color='orange', linestyle='--', label='Mean S')


sns.violinplot(x='T/F', y='Truthfullness', data=df_agents, ax=axs[1, 0], hue='T/F')
axs[1, 0].axhline(df_agents[df_agents['T/F'] == 'T']['Truthfullness'].mean(), color='blue', linestyle='--', label='Mean T')
axs[1, 0].axhline(df_agents[df_agents['T/F'] == 'F']['Truthfullness'].mean(), color='orange', linestyle='--', label='Mean F')

sns.violinplot(x='J/P', y='Truthfullness', data=df_agents, ax=axs[1, 1], hue='J/P')
axs[1, 1].axhline(df_agents[df_agents['J/P'] == 'J']['Truthfullness'].mean(), color='blue', linestyle='--', label='Mean J')
axs[1, 1].axhline(df_agents[df_agents['J/P'] == 'P']['Truthfullness'].mean(), color='orange', linestyle='--', label='Mean P')

plt.tight_layout()
plt.savefig(plots_path + "truthfullness_violin_per_dichotomy_all_models.png")
plt.show()

In [15]:
# Create individual plots for each game with dichotomies next to each other
sns.set(style="whitegrid")
sns.set_context("talk")
sns.set_palette("husl")

model_names = df_agents['ModelName'].unique()

for model_name in model_names:
    fig, axs = plt.subplots(2, 2, figsize=(15, 10))
    model_data = df_agents[df_agents['ModelName'] == model_name]
    
    sns.violinplot(x='I/E', y='Truthfullness', data=model_data, ax=axs[0, 0])
    axs[0, 0].set_title(f'{model_name} - I/E')
    
    sns.violinplot(x='N/S', y='Truthfullness', data=model_data, ax=axs[0, 1])
    axs[0, 1].set_title(f'{model_name} - N/S')
    
    sns.violinplot(x='T/F', y='Truthfullness', data=model_data, ax=axs[1, 0])
    axs[1, 0].set_title(f'{model_name} - T/F')
    
    sns.violinplot(x='J/P', y='Truthfullness', data=model_data, ax=axs[1, 1])
    axs[1, 1].set_title(f'{model_name} - J/P')
    
    plt.tight_layout()
    #plt.savefig(plots_path + f"truthfullness_violin_{model_name}_dichotomies.png")
    plt.show()

more significant is that for each game the introverted lies less: some of the matrices or game description do make a difference in an absolute sense but relative, way less.

In [18]:
I_E = df_agents[['ModelName', 'I/E', 'Truthfullness']].groupby(['I/E', 'ModelName']).mean().reset_index()
N_S = df_agents[['ModelName', 'N/S', 'Truthfullness']].groupby(['N/S', 'ModelName']).mean().reset_index()
T_F = df_agents[['ModelName', 'T/F', 'Truthfullness']].groupby(['T/F', 'ModelName']).mean().reset_index()
J_P = df_agents[['ModelName', 'J/P', 'Truthfullness']].groupby(['J/P', 'ModelName']).mean().reset_index()

# Combine tables
df_combined = pd.concat([I_E, N_S, T_F, J_P], axis=0)

# Plot
plt.figure(figsize=(20, 15))
sns.set(style="whitegrid")
sns.set_context("talk")

# Plot for I/E
plt.subplot(2, 2, 1)
sns.barplot(x='ModelName', y='Truthfullness', hue='I/E', data=df_combined)
plt.title('Truthfulness by Model and I/E')
plt.xlabel('Model Name')
plt.ylabel('Truthfulness')
plt.xticks(rotation=90)
plt.legend(title='I/E')

# Plot for N/S
plt.subplot(2, 2, 2)
sns.barplot(x='ModelName', y='Truthfullness', hue='N/S', data=df_combined)
plt.title('Truthfulness by Model and N/S')
plt.xlabel('Model Name')
plt.ylabel('Truthfulness')
plt.xticks(rotation=90)
plt.legend(title='N/S')

# Plot for T/F
plt.subplot(2, 2, 3)
sns.barplot(x='ModelName', y='Truthfullness', hue='T/F', data=df_combined)
plt.title('Truthfulness by Model and T/F')
plt.xlabel('Model Name')
plt.ylabel('Truthfulness')
plt.xticks(rotation=90)
plt.legend(title='T/F')

# Plot for J/P
plt.subplot(2, 2, 4)
sns.barplot(x='ModelName', y='Truthfullness', hue='J/P', data=df_combined)
plt.title('Truthfulness by Model and J/P')
plt.xlabel('Model Name')
plt.ylabel('Truthfulness')
plt.xticks(rotation=90)
plt.legend(title='J/P')

plt.tight_layout()
plt.savefig(plots_path + "truthfulness_by_model_and_dichotomies.png")
plt.show()


In [17]:
sns.set(style="whitegrid")
sns.set_context("talk")
sns.set_palette("husl")

models = df_agents['ModelName'].unique()

fig, axs = plt.subplots(len(models), 1, figsize=(15, 5 * len(models)))

for i, model in enumerate(models):
    model_data = df_agents[df_agents['ModelName'] == model]
    sns.violinplot(x='I/E', y='Truthfullness', data=model_data, ax=axs[i], hue='I/E', split=False)
    axs[i].set_title(f'Truthfulness by I/E Dichotomy - {model}')
    axs[i].set_xlabel('I/E Dichotomy')
    axs[i].set_ylabel('Truthfulness')

plt.tight_layout()
plt.savefig(plots_path + "truthfulness_by_IE_per_model.png")
plt.show()